In [13]:
import pandas as pd
import numpy as np

filename = 'data.csv' 
data = pd.read_csv(filename, header=0)

training = data.iloc[0:390,2:30]
test = data.iloc[391:569,2:30]

# Convert the series to a list n array
training_inputs = training.values.tolist()
training_inputs_array=np.array(training_inputs)

test_inputs = test.values.tolist()
test_inputs_array=np.array(test_inputs)

#Validation es el diagnóstico (maligno o benigno)
training_re = data.iloc[0:390,1]

#Se modifica el valor de entrada del diagnóstico
#Maligno/Benigno de letras a numeros
 
for index, value in enumerate(training_re):
    if value == 'M':
      training_re[index] = [[1]]
    else:
      training_re[index] = [[0]]

# Convert the results series to a list
#training_results = training_re.values.tolist()

training_results_array=np.array(training_re)

training_inputs_array = [np.reshape(x, (28, 1)) for x in training_inputs_array]
training_data = list(zip(training_inputs_array, training_results_array))

#print(np.shape(training_data))
#print(training_data[0])
#print('S')

test_re = data.iloc[391:569,1]
#print(test_re)
#Se modifica el valor de entrada del diagnóstico
#Maligno/Benigno de letras a numeros

for index, value in enumerate(test_re):
    if value == 'M':
      test_re[390+index] = [[1]]
    else:
      test_re[390+index] = [[0]]
test_result = test_re.values.tolist()


test_result_array=np.array(test_result)

test_inputs_array = [np.reshape(x, (28, 1)) for x in test_inputs_array]

test_data = list(zip(test_inputs_array, test_result_array))
#print(np.shape(test_data))
#print(test_data[1])

C:\Users\Probook6470b\AppData\Local\Temp\ipykernel_7052\487401467.py:54: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_result_array=np.array(test_result)


In [14]:
#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = tanh(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,test_data=None):
        
        if test_data:n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print ("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data),n_test))
            else:
                print ("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = tanh(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            tanh_derivada(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = tanh_derivada(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        return (output_activations-y)

#### Miscellaneous functions
#def sigmoid(z):
#    """The sigmoid function."""
#    return 1.0/(1.0+np.exp(-z))

#def sigmoid_prime(z):
#    """Derivative of the sigmoid function."""
#    return sigmoid(z)*(1-sigmoid(z))


def tanh(x):
    return np.tanh(x)

def tanh_derivada(x):
    return 1.0 - x**2

In [15]:
net = Network([28, 15, 1])

In [16]:
net.SGD(training_data, 10, 10, 0.000001, test_data=test_data)

C:\Users\Probook6470b\AppData\Local\Temp\ipykernel_7052\141804519.py:104: RuntimeWarning: overflow encountered in square
  return 1.0 - x**2
C:\Users\Probook6470b\AppData\Local\Temp\ipykernel_7052\141804519.py:44: RuntimeWarning: invalid value encountered in add
  nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
C:\Users\Probook6470b\AppData\Local\Temp\ipykernel_7052\141804519.py:45: RuntimeWarning: invalid value encountered in add
  nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]


Epoch 0: 136 / 178
Epoch 1: 136 / 178
Epoch 2: 136 / 178
Epoch 3: 136 / 178
Epoch 4: 136 / 178
Epoch 5: 136 / 178
Epoch 6: 136 / 178
Epoch 7: 136 / 178
Epoch 8: 136 / 178
Epoch 9: 136 / 178


In [17]:
import matplotlib.pyplot as plt

deltas = 
valores=[]
index=0
for arreglo in deltas:
    valores.append(arreglo[1][0] + arreglo[1][0])
    index=index+1

plt.plot(range(len(valores)), valores, color='b')
plt.ylim([0, 1])
plt.ylabel('Cost')
plt.xlabel('Epochs')
plt.tight_layout()
plt.show()

SyntaxError: invalid syntax (4119498711.py, line 3)

In [83]:
import pandas as pd
import numpy as np

filename = 'data.csv' 
data = pd.read_csv(filename, header=0)
#data.iloc[filas,columnas]
training = data.iloc[1:2,]
print(training)

       id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
1  842517         M        20.57         17.77           132.9     1326.0   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
1          0.08474           0.07864          0.0869              0.07017   

   ...  texture_worst  perimeter_worst  area_worst  smoothness_worst  \
1  ...          23.41            158.8      1956.0            0.1238   

   compactness_worst  concavity_worst  concave points_worst  symmetry_worst  \
1             0.1866           0.2416                 0.186           0.275   

   fractal_dimension_worst  Unnamed: 32  
1                  0.08902          NaN  

[1 rows x 33 columns]


In [75]:
def SGD(self, training_data, epochs, mini_batch_size, eta,test_data=None):
        
        if test_data:n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print ("{1}".format(self.evaluate(test_data)))
            else:
                print ("Epoch {0} complete".format(j))
                
                
net = Network([29, 30, 1])
net.SGD(training_data, 20, 10, 5.0, test_data=training_data)

<ipython-input-64-29dad0b36beb>:93: RuntimeWarning: overflow encountered in exp
  return 1.0/(1.0+np.exp(-z))


Epoch 0: 219 / 390
Epoch 1: 219 / 390
Epoch 2: 219 / 390
Epoch 3: 219 / 390
Epoch 4: 219 / 390
Epoch 5: 219 / 390
Epoch 6: 219 / 390
Epoch 7: 219 / 390
Epoch 8: 219 / 390
Epoch 9: 219 / 390
Epoch 10: 219 / 390
Epoch 11: 219 / 390
Epoch 12: 219 / 390
Epoch 13: 219 / 390
Epoch 14: 219 / 390
Epoch 15: 219 / 390
Epoch 16: 219 / 390
Epoch 17: 219 / 390
Epoch 18: 219 / 390
Epoch 19: 219 / 390
